<a href="https://colab.research.google.com/github/shrestha94sujan/Image_classification/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print("Hello fren, Welcome to my image classifier!")

Hello fren, Welcome to my image classifier!
